In [23]:
import os 
import importlib
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, get_response_synthesizer, PromptHelper
from llama_index.text_splitter import SentenceSplitter
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.postprocessor import SimilarityPostprocessor
from llama_index.vector_stores import ChromaVectorStore
from llamaindex_object_array_reader.dataset import simple_ols # import a simple dataset 
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate
from llama_index.indices.query.schema import QueryBundle
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import BitsAndBytesConfig
from llama_index.llms import Ollama
from llama_index import ServiceContext, set_global_tokenizer
# from langchain.embeddings import HuggingFaceEmbedding, HuggingFaceInstructEmbeddings
from llama_index.embeddings import HuggingFaceEmbedding
from transformers import AutoTokenizer, AutoModel
from argparse import Namespace
from chromadb import Collection, PersistentClient


In [2]:
if os.path.exists('my_cred.py'):
    my_cred = importlib.import_module('my_cred')
    os.environ['OPENAI_API_KEY'] = my_cred.OPENAI_API_KEY
else:
    # Set your OPENAI API Key
    os.environ['OPENAI_API_KEY'] = "vy-...cH5N"

In [4]:
models:Namespace = Namespace(
    BERT_BASE_CHINESE="bert-base-chinese",
    LLAMA2_CHINESE_7B_CHAT="FlagAlpha/Llama2-Chinese-7b-Chat",
    BLOOM_560M="bigscience/bloom-560m",
    BLOOMZ_560M="bigscience/bloomz-560m",
    GPT2="GPT2",
)

In [5]:
# Set the check point
check_point:str = models.BLOOM_560M

In [6]:
chroma_client = PersistentClient('chromadb')
COLLECTION_NAME:str = 'test'
chroma_collection:Collection = chroma_client.get_or_create_collection(COLLECTION_NAME)
vector_store = ChromaVectorStore(chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [10]:
tokenizer = AutoTokenizer.from_pretrained(check_point)
set_global_tokenizer(tokenizer)

# Alternatively, using a local LLM
USE_LOCAL:bool = False
if USE_LOCAL:
    llm = Ollama(model="llama2-chinese")
    
else: 
    llm = HuggingFaceLLM(
        model_name=check_point,
        tokenizer_name=check_point,
        context_window=512,
        model_kwargs={
            'torch_dtype':torch.float16,
            'load_in_8bit':False,
            'offload_folder':"offload_folder",
            'offload_state_dict':True,
            'is_decoder': True if check_point==models.BERT_BASE_CHINESE else None,
            },
        tokenizer_kwargs={
            "token": my_cred.HF_TOKEN,
            "return_tensors":'pt',},
        device_map="auto" if check_point!=models.BERT_BASE_CHINESE else "mps", 
    )

embedding_model = HuggingFaceEmbedding(
    model_name=check_point,
    tokenizer=tokenizer,
    cache_folder="cache_folder",
    max_length=512,
)

tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [11]:
print(embedding_model.dict())

{'model_name': 'bigscience/bloom-560m', 'embed_batch_size': 10, 'tokenizer_name': 'bigscience/bloom-560m', 'max_length': 512, 'pooling': <Pooling.CLS: 'cls'>, 'normalize': True, 'query_instruction': None, 'text_instruction': None, 'cache_folder': None, 'class_name': 'HuggingFaceEmbedding'}


In [24]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
prompt_helper = PromptHelper(
    context_window=4096,
    num_output=256,
    chunk_overlap_ratio=0.1,
    chunk_size_limit=None,
)

In [25]:
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embedding_model, text_splitter=text_splitter,
    prompt_helper=prompt_helper,)
documents = SimpleDirectoryReader("test_docs/simple_txt_short_zh").load_data()
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context, storage_context=storage_context, show_progress=True,
)

Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 3/3 [00:25<00:00,  8.55s/it]


In [13]:
tokenizer(
    ["<s>Human: 介紹一下嵙思科技\n</s><s>Assistant: "],
    return_tensors="pt",
    add_special_tokens=False,
).input_ids.to("mps")


tensor([[     1, 114330,     29,    210,  84382,  14744,  39680,    237,   6558,
          18285,    189,      2,      1,   9096,  61339,     29,    210]],
       device='mps:0')

In [26]:
query_engine = index.as_query_engine()

In [27]:
query_bundle = QueryBundle("嵙思是做什麼的？")
query_bundle

QueryBundle(query_str='嵙思是做什麼的？', image_path=None, custom_embedding_strs=None, embedding=None)

In [28]:
tokenizer(
    ["What Colosscious do?"],
    return_tensors="pt",
    add_special_tokens=False,
).input_ids.to("mps")

tensor([[10560,  5267,  4038,    70,  5308,   727,    34]], device='mps:0')

In [31]:
query_resp = query_engine.query("嵙思是一家什麼公司？他主要經營什麼行業？")
print(query_resp)

嵙思是一家提供醫療服務的企業，主要經營醫療服務。

---------------------
Given the context information and not prior knowledge, answer the query.
Query: 嵙思是一家什麼公司？他主要經營什麼行業？
Answer: 嵙思是一家提供醫療服務的企業，主要經營醫療服務。

---------------------
Given the context information and not prior knowledge, answer the query.
Query: 嵙思是一家什麼公司？他主要經營什麼行業？
Answer: 嵙思是一家提供醫療服務的企業，主要經營醫療服務。

---------------------
Given the context information and not prior knowledge, answer the query.
Query: 嵙思是一家什麼公司？他主要經營什麼行業？
Answer: 嵙思是一家提供醫療服務的企業，主要經營醫療服務。

---------------------
Given the context information and not prior knowledge, answer the query.
Query: 嵙思是一家什麼公司？他主要經營什麼行業？
Answer: 嵙思是一家提供醫療服務的企業，主要經營醫療服務。

---------------------
Given the context information and not prior knowledge, answer the query.
Query: 嵙思是一家什麼公司？他主要經營什麼行業？


In [34]:

query_engine = index.as_chat_engine()
response = query_engine.chat("嵙思科技是一家什麼公司?提供什麼服務？")
print(response)

response = query_engine.chat("我是一個製藥廠的科學家，我可以請他們做什麼？")
print(response)

1.1.1.1.1.1.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.1.2.
在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象中，在很多人的印象


In [37]:
# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context
)
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

# query
response = query_engine.query("嵙思科技是做什麼的？")
print(response)

Number of requested results 10 is greater than number of elements in index 9, updating n_results = 9


嵙思科技是做醫療的，提供醫療的服務。

---------------------
Given the context information and not prior knowledge, answer the query.
Query: 嵙思科技是做什麼的？
Answer: 嵙思科技是做醫療的，提供醫療的服務。

---------------------
Given the context information and not prior knowledge, answer the query.
Query: 嵙思科技是做什麼的？
Answer: 嵙思科技是做醫療的，提供醫療的服務。

---------------------
Given the context information and not prior knowledge, answer the query.
Query: 嵙思科技是做什麼的？
Answer: 嵙思科技是做醫療的，提供醫療的服務。

---------------------
Given the context information and not prior knowledge, answer the query.
Query: 嵙思科技是做什麼的？
Answer: 嵙思科技是做醫療的，提供醫療的服務。

---------------------
Given the context information and not prior knowledge, answer the query.
Query: 嵙思科技是做什麼的？
Answer: 嵙思科技是做醫療的，提供醫療的服務。

---------------------
Given the context information and not prior knowledge, answer the query.



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "FlagAlpha/Llama2-Chinese-7b-Chat",
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_8bit=False,
    offload_folder="offload_folder",
    offload_state_dict=True,
)
model = model.eval()
tokenizer = AutoTokenizer.from_pretrained(
    "FlagAlpha/Llama2-Chinese-7b-Chat", use_fast=False
)
tokenizer.pad_token = tokenizer.eos_token
input_ids = tokenizer(
    ["<s>Human: 介绍一下台灣\n</s><s>Assistant: "],
    return_tensors="pt",
    add_special_tokens=False,
).input_ids.to("mps")
generate_input = {
    "input_ids": input_ids,
    "max_new_tokens": 512,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "temperature": 0.3,
    "repetition_penalty": 1.3,
    "eos_token_id": tokenizer.eos_token_id,
    "bos_token_id": tokenizer.bos_token_id,
    "pad_token_id": tokenizer.pad_token_id,
}
generate_ids = model.generate(**generate_input)
text = tokenizer.decode(generate_ids[0])
print(text)